# Suicide analysis
### Lukas Forst

In [12]:
import pandas as pd
import numpy as np
import plotly as plt
import seaborn as sns

Now let's look at the data.

In [3]:
data = pd.read_csv('data.csv')
data.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


Now, select random sample from the dataset.

In [4]:
data.sample(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
20150,Qatar,2010,male,35-54 years,18,523563,3.44,Qatar2010,0.844,"125,122,306,346",74055,Generation X
11713,Iceland,1988,male,25-34 years,4,21500,18.60,Iceland1988,NaN,"6,016,168,896",26249,Boomers
26795,United Kingdom,2011,female,25-34 years,134,4094631,3.27,United Kingdom2011,0.901,"2,619,700,404,733",44491,Millenials
5043,Canada,1995,female,25-34 years,141,2431800,5.80,Canada1995,0.861,"604,031,623,433",21871,Generation X
451,Antigua and Barbuda,2002,male,25-34 years,0,6921,0.00,Antigua and Barbuda2002,NaN,"814,615,333",10499,Generation X


In [5]:
data.describe()

,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
count,27820.000000,27820.000000,2.782000e+04,27820.000000,8364.000000,27820.000000
mean,2001.258375,242.574407,1.844794e+06,12.816097,0.776601,16866.464414
std,8.469055,902.047917,3.911779e+06,18.961511,0.093367,18887.576472
min,1985.000000,0.000000,2.780000e+02,0.000000,0.483000,251.000000
25%,1995.000000,3.000000,9.749850e+04,0.920000,0.713000,3447.000000
50%,2002.000000,25.000000,4.301500e+05,5.990000,0.779000,9372.000000
75%,2008.000000,131.000000,1.486143e+06,16.620000,0.855000,24874.000000
max,2016.000000,22338.000000,4.380521e+07,224.970000,0.944000,126352.000000


Now as we saw some null values in the `data.head()`, we should probably check, how many null values we have.

In [6]:
rows, cols = data.shape
nulls_sum = data.isnull().sum()
nulls_sum.apply(lambda x: x / rows)

country               0.000000
year                  0.000000
sex                   0.000000
age                   0.000000
suicides_no           0.000000
population            0.000000
suicides/100k pop     0.000000
country-year          0.000000
HDI for year          0.699353
 gdp_for_year ($)     0.000000
gdp_per_capita ($)    0.000000
generation            0.000000
dtype: float64

Now we can see, that for the feature `HDI for year`, we are missing almost 70% of the data. Therefore, we should probably remove this metric.

In [7]:
data = data.drop('HDI for year', axis=1)

In [11]:
data.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,"2,156,624,900",796,Boomers


Let's visualise yearly values.

In [38]:
yearly =  data[['year', 'suicides_no', 'population']].groupby('year').agg({'population':'sum', 'suicides_no':'sum'})
yearly.describe()

,population,suicides_no
count,3.200000e+01,32.000000
mean,1.603817e+09,210888.125000
std,3.930311e+08,55287.550729
min,1.321019e+08,15603.000000
25%,1.520310e+09,202235.000000
50%,1.740078e+09,233384.500000
75%,1.845573e+09,243501.250000
max,1.997297e+09,256119.000000


It is interesting that the `min` and `max` value are so different - like a rank different. Let's investigate.

In [50]:
yearly[yearly['population'] == yearly['population'].min()]

,population,suicides_no
year,,
2016,132101896,15603


It seems that the last year of the dataset is missing some data, so we should remove it in order to work only with complete data.

In [52]:
yearly = yearly.query('year != 2016')

In [63]:
import plotly.graph_objects as go
fig = go.Figure()

# add summed suicides
fig.add_trace(go.Scatter(x = yearly.index.values, y = yearly['suicides_no'], mode = 'lines', name = 'Suicides'))
fig.add_trace(go.Scatter(x = yearly.index.values, y = yearly['population'] / 10000, mode = 'lines', name = 'Population'))

fig.show()
